# Processing raw XMM-Newton Pointed data

This tutorial will teach you how to use DAXA to process raw XMM-Newton data into a science ready state using one line of Python code (or several lines, if you wish to have more control over the settings for each step). **This relies on there being an initialised (either manually before launching Python, or in your bash profile/rc) backend installation of the XMM Science Analysis System (SAS), including accessible calibration files** - DAXA will check for such an installation, and will not allow processing to start without it.

## Import Statements

## An Archive to be processed

Every processing function implemented in DAXA takes an Archive instance as its first argument; if you don't already know what that is then you should go back and read the following tutorials:

* [Creating a DAXA archive](archives.html) - This explains how to create an archive, load an existing archive, and the various properties and features of DAXA archives.
* [Using DAXA missions](missions.html) - Here we explain what DAXA mission classes are and how to use them to select only the data you need.

## One-line solution

## Different XMM processing steps

### EPIC Cameras (CCD)

### Reflection Grating Spectrometer (RGS)

## 